# Lookalike Model

In [39]:
# %pip install pandas
# %pip install numpy
# %pip install scikit-learn
# %pip install seaborn

In [40]:
import pandas as pd
import numpy as np

#### Reading the csv files from Data Folder

In [41]:
customers_df = pd.read_csv('./Data/Customers.csv')
products_df  = pd.read_csv('./Data/Products.csv')
transactions_df = pd.read_csv('./Data/Transactions.csv')

In [42]:
customers_df.head()

,CustomerID,CustomerName,Region,SignupDate
0,C0001,Lawrence Carroll,South America,2022-07-10
1,C0002,Elizabeth Lutz,Asia,2022-02-13
2,C0003,Michael Rivera,South America,2024-03-07
3,C0004,Kathleen Rodriguez,South America,2022-10-09
4,C0005,Laura Weber,Asia,2022-08-15


In [43]:
products_df.head()

,ProductID,ProductName,Category,Price
0,P001,ActiveWear Biography,Books,169.30
1,P002,ActiveWear Smartwatch,Electronics,346.30
2,P003,ComfortLiving Biography,Books,44.12
3,P004,BookWorld Rug,Home Decor,95.69
4,P005,TechPro T-Shirt,Clothing,429.31


In [44]:
transactions_df.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68


In [45]:
# Merging the dataframes
tx_products = transactions_df.merge(
    products_df,
    on='ProductID',
    how='left'
)

In [46]:
tx_products.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,ProductName,Category,Price_y
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68


In [47]:
# merging the customers to get the regions and signup infomations
tx_full = tx_products.merge(
    customers_df,
    on='CustomerID',
    how='left'
)

In [48]:
tx_full.head()

,TransactionID,CustomerID,ProductID,TransactionDate,Quantity,TotalValue,Price_x,ProductName,Category,Price_y,CustomerName,Region,SignupDate
0,T00001,C0199,P067,2024-08-25 12:38:23,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Andrea Jenkins,Europe,2022-12-03
1,T00112,C0146,P067,2024-05-27 22:23:54,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Brittany Harvey,Asia,2024-09-04
2,T00166,C0127,P067,2024-04-25 07:38:55,1,300.68,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Kathryn Stevens,Europe,2024-04-04
3,T00272,C0087,P067,2024-03-26 22:55:37,2,601.36,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Travis Campbell,South America,2024-04-11
4,T00363,C0070,P067,2024-03-21 15:10:10,3,902.04,300.68,ComfortLiving Bluetooth Speaker,Electronics,300.68,Timothy Perez,Europe,2022-03-15


In [49]:
# converting the date to datetime
tx_full['TransactionDate'] = pd.to_datetime(tx_full['TransactionDate'])
tx_full['SignupDate'] = pd.to_datetime(tx_full['SignupDate'])

In [50]:
tx_full['SignupYear']  = tx_full['SignupDate'].dt.year
tx_full['SignupMonth'] = tx_full['SignupDate'].dt.month

In [51]:
# we want to aggrefate these features again to compte the following:
'''
1. Total cost
2. no. of transactions
3. avg price of purchase
4. distribution of purchases acc cat
5. region
6. signup details
'''
customer_agg = tx_full.groupby('CustomerID').agg({
    'TotalValue': 'sum',
    'TransactionID': 'count',
    'Price_x': 'mean',
    'Category': lambda x: x.value_counts(normalize=True).to_dict(),
    'Region': 'first',
    'SignupYear': 'first',
    'SignupMonth': 'first'
}).rename(columns={
    'TotalValue': 'TotalSpend',
    'TransactionID': 'TransactionCount',
    'Price_x': 'AvgPrice'
}).reset_index()



In [52]:
customer_agg.head()

,CustomerID,TotalSpend,TransactionCount,AvgPrice,Category,Region,SignupYear,SignupMonth
0,C0001,3354.52,5,278.334000,"{'Electronics': 0.6, 'Books': 0.2, 'Home Decor...",South America,2022,7
1,C0002,1862.74,4,208.920000,"{'Home Decor': 0.5, 'Clothing': 0.5}",Asia,2022,2
2,C0003,2725.38,4,195.707500,"{'Home Decor': 0.5, 'Clothing': 0.25, 'Electro...",South America,2024,3
3,C0004,5354.88,8,240.636250,"{'Books': 0.375, 'Home Decor': 0.375, 'Electro...",South America,2022,10
4,C0005,2034.24,3,291.603333,"{'Electronics': 0.6666666666666666, 'Home Deco...",Asia,2022,8


#### Converting categorical distrivution into seeratecolumns for each categorical 

In [53]:
all_categories = ['Books','Electronics','Clothing','Home Decor']

In [54]:
for cat in all_categories:
    col_name = cat + 'Pct'
    customer_agg[col_name] = customer_agg['Category'].apply(
        lambda d: d.get(cat, 0)
    )

In [55]:
# remove the original category column since we have the percentages
customer_agg.drop(columns=['Category'], inplace=True)

Encode Categorical Features (Region) and Scale

In [56]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [57]:
le_region = LabelEncoder()
customer_agg['RegionEnc'] = le_region.fit_transform(customer_agg['Region'])

In [58]:
numeric_cols = [
    'TotalSpend', 'TransactionCount', 'AvgPrice',
    'BooksPct','ElectronicsPct','ClothingPct','Home DecorPct',
    'SignupYear','SignupMonth','RegionEnc'
]


In [59]:
scaler = StandardScaler()
customer_agg_scaled = customer_agg.copy()
customer_agg_scaled[numeric_cols] = scaler.fit_transform(customer_agg_scaled[numeric_cols])

#### Feature matrix for each customer then compute the similarities


In [60]:
feature_matrix = customer_agg_scaled[numeric_cols].values

In [61]:
from sklearn.metrics.pairwise import cosine_similarity

In [62]:
# Calculate cosine similarity among all customers
sim_matrix = cosine_similarity(feature_matrix, feature_matrix)

In [63]:
# Prepare a mapping from CustomerID to row index
cust_ids = customer_agg_scaled['CustomerID'].tolist()
cust_id_to_idx = {cid: idx for idx, cid in enumerate(cust_ids)}

#### 5. For Each Customer (C0001 – C0020), Find Top 3 Lookalikes

In [64]:
lookalike_results = []
target_customers = [f'C00{str(i).zfill(2)}' for i in range(1, 21)]

In [65]:
# we are trying to look for first 20 customers
for cust_id in target_customers:
    if cust_id not in cust_id_to_idx:
        continue
        # ignoring the customers that are not in the dataset

    idx = cust_id_to_idx[cust_id]
    cust_sim_scores = sim_matrix[idx]

    # sort them in descending order
    top_indexes = np.argsort(-cust_sim_scores)

    lookalikes = []
    for neighbor_idx in top_indexes:
        if neighbor_idx == idx: 
            continue
        if len(lookalikes) < 3:
            neighbor_cust_id = cust_ids[neighbor_idx]
            neighbor_score   = cust_sim_scores[neighbor_idx]
            lookalikes.append((neighbor_cust_id, neighbor_score))
        else:
            break
    lookalike_results.append({
        'CustomerID': cust_id,
        'LookalikeList': lookalikes
    })
        



In [67]:
# creating the csv file
rows_for_csv = []
for entry in lookalike_results:
    cid = entry['CustomerID']
    # lookalikes_str = ';'.join([f"{lk[0]}|{lk[1]:.4f}" for lk in entry['LookalikeList']])
    # store in following format: (Lookalike1,Score1),(Lookalike2,Score2),(Lookalike3,Score3)
    lookalikes_str = ','.join([f"({lk[0]},{lk[1]:.4f})" for lk in entry['LookalikeList']])
    rows_for_csv.append({
        'CustomerID': cid,
        'Lookalikes': lookalikes_str
    })

lookalikes_df = pd.DataFrame(rows_for_csv)
lookalikes_df.to_csv('Sangam_Lookalike.csv', index=False)
lookalikes_df.head(20)

,CustomerID,Lookalikes
0,C0001,"(C0112,0.8277),(C0192,0.7696),(C0120,0.7136)"
1,C0002,"(C0134,0.9561),(C0166,0.8368),(C0159,0.8057)"
2,C0003,"(C0031,0.9630),(C0129,0.8592),(C0189,0.8128)"
3,C0004,"(C0113,0.8724),(C0102,0.7795),(C0122,0.7180)"
4,C0005,"(C0007,0.9530),(C0140,0.8759),(C0199,0.8343)"
5,C0006,"(C0126,0.8097),(C0137,0.7955),(C0024,0.6874)"
6,C0007,"(C0005,0.9530),(C0140,0.8378),(C0197,0.7190)"
7,C0008,"(C0194,0.8190),(C0024,0.7478),(C0081,0.6413)"
8,C0009,"(C0150,0.8009),(C0092,0.7894),(C0058,0.7442)"
9,C0010,"(C0062,0.8625),(C0034,0.8335),(C0077,0.8201)"
